# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [2]:
from pathlib import Path
import sys

nb_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
parent = nb_dir.parent  # points to 03-building-agents
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

In [3]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [4]:
load_dotenv()

False

## Play with Tavily

In [5]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [6]:
# [TODO] Define a query and run
query = "what is solana"
result = client.search(query)

In [7]:
result

{'query': 'what is solana',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://solana.com/learn/what-is-solana',
   'title': 'What is Solana?',
   'content': 'Solana is a fast, low-cost, high-performance network enabling secure digital transactions and powering applications, like the internet for value transfer.',
   'score': 0.9387167,
   'raw_content': None},
  {'url': 'https://www.forbes.com/sites/digital-assets/article/what-is-solana-sol-how-it-works-and-what-to-know/',
   'title': 'What Is Solana (SOL)? How It Works And What To Know - Forbes',
   'content': 'Solana is a high-performance public blockchain platform that provides fast transaction speeds and low fees for developers and users, especially in comparison to some of the older public blockchains such as Ethereum. The focus on high throughput, potentially handling thousands of transactions per second, has made Solana one of the more ambitious blockchain networks in the industry. Solan

## Define Web Search tool

In [8]:
from lib.tools import web_search, compare_sources

In [9]:
tools = [web_search, compare_sources]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [10]:
# [TODO] Agent with no tools
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful assistant",
)

In [11]:
# [TODO] Agent with web search tool
instructions=(
    "You are a web-aware assistant that can search for updated information. "
    "For each query, you will search the web for current information using " 
    "Tavily's AI-optimized search and provide a comprehensive answer.\n"
    "Always cite your sources and explain any discrepancies found.\n"
    "Be particularly attentive to dates and time-sensitive information."
)
web_agent = Agent(model_name="gpt-4o-mini", instructions=instructions, tools=tools)


In [12]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [13]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events that occurred after October 2021, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most accurate and up-to-date information., tool_calls = None)


In [14]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have information on events that occurred after October 2021, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most accurate and up-to-date information.


In [15]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information on events that occurred after October 2021, including the winners of the 2025 Oscars. You may want to check the latest news or the official Oscars website for the most accurate and up-to-date information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last knowledge update in October 2021, I can't provide the latest developments in AI technology beyond that date. However, I can summarize some key trends and areas of focus in AI leading up t

In [16]:
print(run2.get_final_state()["messages"][-1].content)

As of my last knowledge update in October 2021, I can't provide the latest developments in AI technology beyond that date. However, I can summarize some key trends and areas of focus in AI leading up to that time:

1. **Natural Language Processing (NLP):** Significant advancements have been made with models like GPT-3 (developed by OpenAI) and other large language models, improving capabilities in text generation, translation, and understanding context.

2. **Computer Vision:** AI systems have become increasingly adept at image and video recognition. Applications include facial recognition, autonomous vehicles, and medical imaging.

3. **Ethics and Bias in AI:** There has been a growing awareness and concern regarding the ethical implications of AI, including issues of bias, privacy, and transparency. Companies and researchers are working on frameworks to address these challenges.

4. **AI in Healthcare:** AI technologies have been increasingly applied in healthcare, including drug dis

**Web Agent**

In [17]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a web-aware assistant that can search for updated information. For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer.
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_iKjiVHX2FxU1q7MV1DifJoc7', function=Function(arguments='{"query":"2025 Oscar winner International Movie","search_depth":"basic"}', name='web_search

In [18]:
print(run1.get_final_state()["messages"][-1].content)

The 2025 Oscar for Best International Feature Film was awarded to Brazil's **"I'm Still Here."** The film, directed by Walter Salles and featuring actress Fernanda Torres, is based on a true story set in 1970s Rio de Janeiro during a time of military dictatorship. Torres plays the lead role of Eunice Paiva, and the film also features her mother, the renowned Fernanda Montenegro, in a supporting role.

For more information, you can read the full articles from NPR and ABC covering the Oscars:

- [NPR Article on "I'm Still Here"](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature)
- [ABC's Complete List of Oscar Winners](https://abc.com/news/8d96bb37-bd59-4c28-a470-46f244f63b3b/category/3590742)


In [19]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a web-aware assistant that can search for updated information. For each query, you will search the web for current information using Tavily's AI-optimized search and provide a comprehensive answer.
Always cite your sources and explain any discrepancies found.
Be particularly attentive to dates and time-sensitive information., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_iKjiVHX2FxU1q7MV1DifJoc7', function=Function(arguments='{

In [20]:
print(run2.get_final_state()["messages"][-1].content)

Recent developments in AI technology as of 2025 demonstrate significant advancements across various domains, including efficiency, scientific collaboration, and enhanced reasoning capabilities. Here are some key trends and innovations:

1. **Efficiency and Customization**:
   - Companies are focusing on developing custom silicon to reduce costs related to AI features, such as recommender systems. This includes advancements in continuous learning, which allows AI systems to adapt based on interactions without needing full retraining. This adaptability is expected to contribute to a long-term increase in demand for AI technologies, as noted by experts at Morgan Stanley ([Morgan Stanley](https://www.morganstanley.com/insights/articles/ai-trends-reasoning-frontier-models-2025-tmt)).

2. **Scientific Collaboration**:
   - AI is increasingly acting as a collaborator in scientific research. For instance, systems like DeepMind's Co-Scientist and Stanford's Virtual Lab are capable of autonomous

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow